In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from autoplan.generator import ProgramGenerator
from grammars.pythagorean.program import Program, PythagoreanLabels
from autoplan.dataset import build_synthetic_dataset
from autoplan.trainer import ParserTrainer
from autoplan.parsing import ProgramParser
from autoplan.token import JavaTokenizer
from tqdm import tqdm_notebook as tqdm
import torch
import pandas as pd
from pprint import pprint

# device = torch.device('cpu')
device = torch.device('cuda:0')

In [ ]:
program_grammar = Program()
generator = ProgramGenerator(grammar=program_grammar)
program, choices, choice_options = generator.generate()
print(program)
pprint(choices)
pprint(choice_options)

In [ ]:
dataset = build_synthetic_dataset(
    N_train=1000, 
    N_val=100, 
    tokenizer=JavaTokenizer(),
    generator=ProgramGenerator(grammar=program_grammar))

In [ ]:
dataset.train_dataset[0]

In [ ]:
batch = next(iter(dataset.loader(dataset.val_dataset)))

In [ ]:
trainer = ParserTrainer(program_grammar, dataset, device)

In [ ]:
losses = []
accuracy = []
for _ in tqdm(range(20)):
    losses.append(trainer.train_one_epoch())
    accuracy.append(trainer.eval())
pd.Series(losses).plot()

In [ ]:
trainer.model.save('checkpoint.ph.tar')

In [ ]:
parser = ProgramParser(program_grammar, dataset, 'checkpoint.ph.tar', device)

In [ ]:
parser.infer(batch['program'][:1], batch['program_len'][:1])